IMPORTS

In [2]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
from pathlib import Path  

In [4]:
from scipy.stats import binom,poisson,geom,hypergeom,uniform,expon, gamma, beta,chi2,t,pareto,lognorm

In [5]:
from random import randrange, random
import math

CREACIÓN SERIES TEMPORALES SEGÚN: PERIODOS / FIN

In [6]:
# A partir de una fecha de inicio y fecha de fin, devuelve la serie temporal desde la fecha de inicio a la fecha de fin con frequencias posibles:
# B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds

def series_fin(inicio, fin,freq):
    serie = pd.date_range(start=inicio, end=fin, freq=freq)
    return serie 

In [7]:
# A partir de una fecha de inicio y el número de periodos, devuelve la serie temporal desde la fecha de inicio a la fecha de fin con frequencias posibles:
# B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds

def series_periodos(inicio, periodos, freq): 
    serie = pd.date_range(start=inicio, periods=periodos, freq=freq)
    return serie

MODELOS DE TENDENCIA DETERMINISTA

EN ESTOS MODELOS LAS PREDICCIONES NO SE BASAN EN LA PROPIEDAD QUE LOS DATOS DEPENDEN EN MAYOR GRADO DE LOS DATOS MÁS RECIENTES RESPECTO A DATOS PASADOS. 

MODELOS SIN ESTACIONALIDAD

TRES TIPOS DE TENDENCIA:

In [8]:
def tendencia_lineal (a,b,t):
    return a + b * t 

In [9]:
# El valor de la media crece/decrece de forma cte y siempre en la misma dirección
# Variación cte --> estacionaria respecto a la varianza

def tendencia_determinista_lineal (a,b,t,e=0):
    return tendencia_lineal(a,b,t) + e

In [10]:
# El valor de la media crece/decrece 
# Variación cte en orden k  

def tendencia_determinista_polinómica(params,t,e=0):
    res = params[0]
    for k in range(1,len(params)):
        res = res + params[k] * t**k
    return res + e

In [11]:
# Variación exponencial
def tendencia_determinista_exponencial(a,b,t,e=0):
    return math.exp(a+b*t+e)

In [12]:
#Variación decreciente
def tendencia_determinista_logaritmica(a,b,t,e=0):
    return a + b * math.log(t)

Parámetros del modelo:
    - params: indica los parámetros de la tendencia.
    - tipo: Valores posibles:
        1. Tendencia determinista lineal (2 parámetros)
        2. Tendencia determinista polinómica (n parámetros)
        3. Tendencia determinista exponencial (2 parámetros)
        4. Tendencia determinista logarítmica (2 parámetros)
    - num_datos: número de datos a generar
    - coef_error: indica el coeficiente de error (a mayor coeficiente, menor estacionariedad). Si el coef_error=0, los datos son estacionarios. 

In [13]:
def tendencia_det(params,tipo,num_datos,coef_error=0):
    
    datos = np.zeros(num_datos)
    
    for t in range(1,num_datos+1):
        e = random()*coef_error
        
        if tipo==1:
            datos[t-1] = tendencia_determinista_lineal(params[0],params[1],t,e)
        elif tipo==2:
            datos[t-1] = tendencia_determinista_polinómica(params,t,e)
        elif tipo==3:
            datos[t-1] = tendencia_determinista_exponencial(params[0],params[1],t,e)
        elif tipo==4:
            datos[t-1] = tendencia_determinista_logaritmica(params[0],params[1],t,e)
            
    return datos   

CREACIÓN DE DATOS SEGÚN CIERTAS DISTRIBUCIONES:

Parámetros:
- num_datos : numéro de datos a generar 
- Distr, params : Indica la distribución a seguir y los parámetros de esta distribución. Casos:
    1. Los parámetros son la media y la desv_típica de los datos. Sigue una distribución normal.
    2. Los parámetros son n (nº pruebas) y p (probabilidad éxito) de una distribución binomial. Además, podemos añadir otro parámetro que indica la localización de los datos.
    3. El parámetro es mu de una distribución poisson. Además, podemos añadir otro parámetro que indica la localización de los datos.
    4. El parámetro es p (probabilidad éxito) de una distribución geométrica. Además, podemos añadir otro parámetro que indica la localización de los datos.
    5. Los parámetros son M (tamaño población), n (nº aciertos) y N (tamaño muestra) de una distribución hipergeométrica. Además, podemos añadir otro parámetro que indica la localización de los datos.
    6. El parámetro es el valor constante que toma los datos.
    7. Sigue una distribución uniforme. Puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    8. El parámetro es s de una distribución lognormal. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    9. Sigue una distribución exponencial. Puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    10. El parámetro es a de una distribución gamma. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    11. Los parámetros son a y b de una distribución beta. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    12. El parámetro es df de una distribución chi cuadrado. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    13. El parámetro es t de una distribución t-student. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    14. El parámetro es b de una distribución pareto. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    15. Sigue una distribución lineal descendente cuyos parámetros son el dato inicial y la pendiente (en valor absoluto). El mínimo valor posible es cero, si se alcanza los datos se mantienen constante con ese valor. 
    16. Sigue una distribución lineal ascendente cuyos parámetros son el dato inicial y la pendiente. 
    17. Obtienes números aleatorios cuyos valores oscilan entre el primer parámetro y el segundo.

In [14]:
def crear_datos(distr,params,num_datos):
    
    if distr == 1 :
        datos = np.random.normal(params[0],params[1],num_datos)
        
    elif distr ==2 :
        if len(params)==2:
            datos = binom.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = binom.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr== 3 :
        if len(params)==1:
            datos = poisson.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = poisson.rvs(params[0],params[1],size=num_datos)
            
    elif distr == 4 :
        if len(params)==1:
            datos = geom.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = geom.rvs(params[0],params[1],size=num_datos)
            
    elif distr == 5:
        if len(params)==3:
            datos = hypergeom.rvs(params[0],params[1],params[2],size=num_datos)
        elif len(params) == 4:
            datos = hypergeom.rvs(params[0],params[1],params[2],params[3],size=num_datos)
            
    elif distr == 6: 
        datos = np.zeros(num_datos) + params[0]
        
    elif distr == 7:
        if len(params)==0:
            datos = uniform.rvs(size=num_datos)
        elif len(params)==1:
            datos = uniform.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = uniform.rvs(params[0],params[1],size=num_datos)
            
    elif distr == 8:
        if len(params)==1:
            datos = lognorm.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = lognorm.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = lognorm.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 9: 
        if len(params)==0:
            datos = expon.rvs(size=num_datos)
        elif len(params)==1:
            datos = expon.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = expon.rvs(params[0],params[1],size=num_datos)
            
    elif distr == 10: 
        if len(params)==1:
            datos = gamma.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = gamma.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = gamma.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 11: 
        if len(params)==2:
            datos = beta.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = beta.rvs(params[0],params[1],params[2],size=num_datos)
        elif len(params) == 4:
            datos = beta.rvs(params[0],params[1],params[2],params[3],size=num_datos)
            
    elif distr == 12: 
        if len(params)==1:
            datos = chi2.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = chi2.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = chi2.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 13: 
        if len(params)==1:
            datos = t.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = t.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = t.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 14: 
        if len(params)==1:
            datos = pareto.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = pareto.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = pareto.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 15:
        datos = np.zeros(num_datos)
        datos[0]= params[0]
        i=1
        while datos[i-1]>0 and i<num_datos:
            datos[i] = datos[i-1] - params[1]
            i= i+1
            
    elif distr == 16:
        datos = np.zeros(num_datos)
        datos[0] = params[0]
        for i in range(1,num_datos):
            datos[i] = datos[i-1] + params[1]
    
    elif distr == 17:
        datos= np.zeros(num_datos)
        for i in range(0,num_datos):
            datos[i] = randrange(params[0],params[1])
        
    return datos

MODELOS PERIÓDICOS

CREACIÓN DE MODELOS DE DATOS PERIÓDICOS A PARTIR DE UNA DISTRIBUCIÓN SABIENDO EL NÚMERO DE DATOS A GENERAR Y ALGUNA DE LAS SIGUIENTES OPCIONES:
    1. AMPLITUD PERIODO
    2. NÚMERO DE PERIODOS

Parámetros:
    - distr, params : indica la distribución que van a seguir los datos en cada periodo y los parámetros de esta distribución.
    - num_datos: indica el número de datos a crear.
    - periodo: indica el número de datos de cada periodo.

In [22]:
def datos_periodicos_amplitud(distr,params,num_datos,periodo):
    
    num_periodos = int(num_datos/periodo)
    res = num_datos % periodo
    datos_base0 = crear_datos(distr,params,periodo)
    datos_base = datos_base0
    for i in range(0,num_periodos-1):
        datos_base=np.concatenate((datos_base0,datos_base))
    if res>0:
        datos_base=np.concatenate((datos_base,datos_base0[:res]))
    return datos_base

In [30]:
datos = datos_periodicos_cantidad(1,[20,2],105,10)

In [31]:
datos

array([23.73575451, 18.22572996, 18.40859707, 24.67779672, 21.88876534,
       17.82684036, 19.74539577, 19.34277029, 18.09009883, 17.92174357,
       23.73575451, 18.22572996, 18.40859707, 24.67779672, 21.88876534,
       17.82684036, 19.74539577, 19.34277029, 18.09009883, 17.92174357,
       23.73575451, 18.22572996, 18.40859707, 24.67779672, 21.88876534,
       17.82684036, 19.74539577, 19.34277029, 18.09009883, 17.92174357,
       23.73575451, 18.22572996, 18.40859707, 24.67779672, 21.88876534,
       17.82684036, 19.74539577, 19.34277029, 18.09009883, 17.92174357,
       23.73575451, 18.22572996, 18.40859707, 24.67779672, 21.88876534,
       17.82684036, 19.74539577, 19.34277029, 18.09009883, 17.92174357,
       23.73575451, 18.22572996, 18.40859707, 24.67779672, 21.88876534,
       17.82684036, 19.74539577, 19.34277029, 18.09009883, 17.92174357,
       23.73575451, 18.22572996, 18.40859707, 24.67779672, 21.88876534,
       17.82684036, 19.74539577, 19.34277029, 18.09009883, 17.92

Parámetros:
    - distr, params : indica la distribución que van a seguir los datos en cada periodo y los parámetros de esta distribución.
    - num_datos: indica el número de datos a crear.
    - num_periodos: indica el número de periodos

In [27]:
def datos_periodicos_cantidad(distr,params,num_datos,num_periodos):
    periodo = int(num_datos/num_periodos)
    res = num_datos % num_periodos
    datos_base0 = crear_datos(distr,params,periodo)
    datos_base = datos_base0
    for i in range(0,num_periodos-1):
        datos_base=np.concatenate((datos_base0,datos_base))
    if res>0:
        datos_base=np.concatenate((datos_base,datos_base0[:res]))
    return datos_base

MODELOS AUTORREGRESIVOS

Parámetros:
    - c (constante), phi (parámetro autorregresivos), a (ruido blanco) : parámetros del modelo autorregresivo.
    - num_datos: numéro de datos a crear 
    - desv: necesario para la creación de datos de a ,en el caso de que pasemos como parámetro un array vacío. Indica la desviación del ruido blanco

In [16]:
# Si orden = 1, el modelo es estacionario si el valor absoluto de phi es menor que 1
# Si orden = 2, el modelo es estacionario si las racies del polinomio autorregresivo son 1 - phi[1] * x - phi[2] * x **2 = 0 en módulo/valor absoluto son mayores que uno

def modelo_AR(c,phi,num_datos,desv,a=[]):
    
    orden = len(phi)
    
    if len(a)==0: 
        a = crear_datos(1,[0,desv],num_datos)
    
    datos = np.zeros(num_datos)
    
    for k in range(0,orden):
        datos[k] = c + a[k]
            
    for i in range(orden,num_datos):
        datos [i]= c + a[i]
        for j in range (1,orden+1):
            datos[i] = datos[i] + phi[j-1]*datos[i-j]
    
    return datos

MODELOS DE MEDIAS MOVILES

Parámetros:

    - c (constante), teta (parámetros modelo), a (ruido blanco) : parámetros del modelo de medias móviles.
    - num_datos: numéro de datos a crear 
    - desv: necesarios para la creación de a, en el caso de que pasemos como parámetro un array vacío. Indica la desviación del ruido blanco

In [17]:
# Modelo siempre estacionario 

def modelo_MA(c,teta,num_datos,desv,a=[]):
    
    orden = len(teta)
    
    if len(a)==0:  
        a = crear_datos(1,[0,desv],num_datos)
    
    datos = np.zeros(num_datos)
    for i in range(0,orden):
        datos[i]= c + a[i]
            
    for i in range(orden,num_datos):
        datos[i] = c + a[i]
        for j in range (1,orden+1):
            datos[i]= datos[i] + teta[j-1]*a[i-j]
            
    return datos

MODELO ARMA 

Parámetros:
    - c (constante), phi, teta (parámetros del modelo), a (ruido blanco) : parámetros del modelo de autorregresivo y de medias móviles.
    - num_datos: numéro de datos a crear 
    - desv: necesarios para la creación de a, en el caso de que pasemos como parámetro un array vacío. Indica la desviación del ruido blanco

Condiciones de estacionariedad iguales a las del modelo AR

In [18]:
def modelo_ARMA(c,phi,teta,num_datos,desv,a=[]):
    
    if len(a)==0:  
        a = crear_datos(1,[0,desv],num_datos)
    
    datos = np.zeros(num_datos)
    p = len(phi)
    q = len(teta)   
    maxi = max(p,q)
    
    for k in range(0,maxi):
        datos[k] = c + a[k]
            
    for i in range(maxi,num_datos):
        datos[i] = c + a[i]
            
        for j in range (1,p+1):
            datos[i]= datos[i] + phi[j-1]*datos[i-j]
                
        for k in range(1,q+1):
            datos[i] = datos[i] + teta[k-1]*a[i-k]   
                
    return datos
    

MODELOS ESTACIONALES 

AUTORREGRESIVOS

Parámetros:
    - c (constante), phi (parámetro autorregresivos), a (ruido blanco) : parámetros del modelo autorregresivo. 
    - s: estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - desv: necesarios para la creación de a, en el caso de que pasemos como parámetro un array vacío. Indica la desviación del ruido blanco

In [19]:
def modelo_AR_estacional(c,phi,s,num_datos,desv,a=[]):
    
    if len(a)==0: 
        a = crear_datos(1,[0,desv],num_datos)
    
    datos = np.zeros(num_datos)
    orden = len(phi)
    
    for k in range(0,s*orden):
        datos[k] = a[k] + c
            
    for i in range(orden*s,num_datos):
        datos [i]= c + a[i]
        for j in range (1,orden+1):
            datos[i] = datos[i] + phi[j-1]*datos[i-j*s]
    
    return datos

MEDIAS MÓVILES

Parámetros:
    - c (constante), teta (parámetrosmodelo), a (ruido blanco) 
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - desv: necesarios para la creación de a en el caso de que no lo pasemos como parámetro. Indica la desviación desviación que presenta el ruido blanco

In [20]:
def modelo_MA_estacional(c,teta,s,num_datos,desv,a=[]):
    
    if len(a)==0: 
        a = crear_datos(1,[0,desv],num_datos)
    
    datos = np.zeros(num_datos)
    orden = len(teta)
    
    for i in range(0,s*orden):
        datos[i]= a[i] + c
            
    for i in range(s*orden,num_datos):
        datos[i] = c + a[i]
        for j in range (1,orden+1):
            datos[i]= datos[i] + teta[j-1]*a[i-j*s]
                
    return datos

ARMA

Parámetros:
    - c (constante), phi, teta (parámetros del modelo), a (ruido blanco): parámetros del modelo de autorregresivo y de medias móviles.
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - desv: necesarios para la creación de a, en el caso de que no lo pasemos como parámetro. Indica la desviación que presenta el ruido blanco

In [21]:
def modelo_ARMA_estacional(c,phi,teta,s,num_datos,desv,a=[]):
    
    if len(a)==0:  
        a = crear_datos(1,[0,desv],num_datos)
    
    datos = np.zeros(num_datos)
    p = len(phi)
    q = len(teta)
    maxi = max(p,q)
    
    for k in range(0,s*maxi):
        datos[k] = a[k] + c
            
    for i in range(s*maxi,num_datos):
        datos[i] = c + a[i]
            
        for j in range (1,p+1):
            datos[i]= datos[i] + phi[j-1]*datos[i-j*s]
                
        for k in range(1,q+1):
            datos[i] = datos[i] + teta[k-1]*a[i-k*s]   
                
    return datos

Parámetros:
    - c (constante), phi, teta (parámetros del modelo), a (ruido blanco): parámetros del modelo de autorregresivo y de medias móviles.
    Casos: 
        1. len(phi)=0 : modelo de medias móviles.
        2. len(teta)=0 : modelo autorregresivos.
        3. len(phi)>0 y len(teta)>0 : modelo ARMA.
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación). Casos:
        1. s>0: modelo estacional.
        2. s=0: modelo no estacional
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de a en el caso de que pasemos como parámetro un array vacío. Indica la distribución con la que se inicializará a y los parámetros de esta distribución.

In [22]:
def creacion_modelos_ARMA(c,num_datos,desv,s=0,phi=[],teta=[],a=[]):
    
    p = len(phi)
    q = len(teta)
    
    if s == 0:
        if q == 0:
            datos = modelo_AR(c,phi,num_datos,desv,a)
            
        elif p == 0: 
            datos = modelo_MA(c,teta,num_datos,desv,a)
        else:
            datos = modelo_ARMA(c,phi,teta,num_datos,desv,a)
    else :
        if q == 0:
            datos = modelo_AR_estacional(c,phi,s,num_datos,desv,a)
            
        elif p == 0: 
            datos = modelo_MA_estacional(c,teta,s,num_datos,desv,a)
        else:
            datos = modelo_ARMA_estacional(c,phi,teta,s,num_datos,desv,a)
            
    return datos 

DRIFT CAMBIO DISTRIBUCIONES:

TIPOS DE DRIFT:
    1. CAMBIO DE DISTRIBUCIONES CREADAS CON CREAR_DATOS
    2. CAMBIO DE DISTRIBUCIÓN CREAR_DATOS A MODELOS ARMA
    3. CAMBIO DE DISTRIBUCIÓN CREAR_DATOS A PERIÓDICOS
    4. CAMBIO DE DISTRIBUCIÓN CREAR_DATOS A TENDENCIAS DETERMINISTAS 
    5. CAMBIO DE MODELO ARMA A MODELO ARMA
    6. CAMBIO DE MODELO ARMA A DISTRIBUCIÓN CREAR DATOS 
    7. CAMBIO DE MODELO ARMA A PERIODICOS 
    8. CAMBIO DE MODELO ARMA A TENDENCIAS DETERMINISTAS
    9. CAMBIO DE PERIÓDICOS A PERIÓDICOS
    10. CAMBIO DE PERIÓDICOS A DISTRIBUCIONES DE CREAR_DATOS
    11. CAMBIO DE PERIODICOS A MODELOS ARMA
    12. CAMBIO DE PERIÓDICOS A TENDENCIAS DETERMINISTAS
    13. CAMBIO DE TENDENCIAS DETERMINISTAS A TENDENCIAS DETERMINISTAS 
    14. CAMBIO DE TENDENCIAS DETERMINISTAS A DISTRIBUCIONES CON CREAR_DATOS
    15. CAMBIO DE TENDENCIAS DETERMINISTAS A MODELOS ARMA
    16. CAMBIO DE TENDENCIAS DETERMINISTAS A PERIÓDICOS

Parámetros:
    - params1,params2 : parámetros necesarios para crear los datos pervios y posteriores al drift. Casos:
        1. DISTRIBUCIONES CREADAS CON CREAR DATOS: los parámetros son distr y parametros que indica la distribución y los parámetros de la distribución.
        2. MODELOS ARMA: los parámetros son los del modelo arma, es decir, c, phi, teta, a, s y desv.
        3. DATOS PERIODICOS: los parametros son el tipo (1 indica que se ha pasado la amplitud del periodo y 2 la cantidad de periodos) y distr, params y p (indica el número de periodos o su amplitud) necesarios para crear los datos periódicos con periodo p / número de periodos p siguiendo una distribución con paramétros params.
        4. TENDENCIA DETERMINISTA: los parámetros son el tipo de tendencia, los coeficientes del modelo de tendencia determinista y el coeficiente de error.  
    - tipo: tipo de drift.
    - num_drift: número de datos previos al drift.
    - num_datos: número de datos totales.

In [23]:
def crear_drift(params1,params2,tipo,num_drift,num_datos):
    
    if tipo==1:
        distr1=params1[0]
        parametros1=params1[1]
        datos1 = crear_datos(distr1,parametros1,num_drift)
        
        distr2=params2[0]
        parametros2=params2[1]
        datos2 = crear_datos(distr2,parametros2,num_datos-num_drift)
        
    elif tipo==2:
        distr1=params1[0]
        parametros1=params1[1]
        datos1 = crear_datos(distr1,parametros1,num_drift)
        
        c = params2[0]
        desv = params2[1]
        s = params2[2]
        phi = params2[3]
        teta = params2[4]
        a = params2[5]
        datos2 = creacion_modelos_ARMA(c,num_datos-num_drift,desv,s,phi,teta,a)
    
    elif tipo==3:
        distr1=params1[0]
        parametros1=params1[1]
        datos1 = crear_datos(distr1,parametros1,num_drift)
        
        tipo2 = params2[0]
        distr,parametros2,p = params2[1],params2[2],params2[3]
        if tipo2==1:
            datos2=datos_periodicos_amplitud(distr,parametros2,num_datos-num_drift,p)
        elif tipo2==2:
            datos2=datos_periodicos_cantidad(distr,parametros2,num_datos-num_drift,p)
            
    elif tipo==4:
        distr1=params1[0]
        parametros1=params1[1]
        datos1 = crear_datos(distr1,parametros1,num_drift)
        
        parametros2,tipo2,coef_error = params2[0],params2[1],params2[2]
        datos2=tendencia_det(parametros2,tipo2,num_datos-num_drift,coef_error)
    
    elif tipo==5:
        c = params1[0]
        desv = params1[1]
        s = params1[2]
        phi = params1[3]
        teta = params1[4]
        a = params1[5]
        datos1 = creacion_modelos_ARMA(c,num_drift,desv,s,phi,teta,a)
        
        c2 = params2[0]
        desv2 = params2[1]
        s2 = params2[2]
        phi2 = params2[3]
        teta2 = params2[4]
        a2 = params2[5]
        datos2 = creacion_modelos_ARMA(c2,num_datos-num_drift,desv2,s2,phi2,teta2,a2)
        
    elif tipo==6: 
        c = params1[0]
        desv = params1[1]
        s = params1[2]
        phi = params1[3]
        teta = params1[4]
        a = params1[5]
        datos1 = creacion_modelos_ARMA(c,num_drift,desv,s,phi,teta,a)
        
        distr2=params2[0]
        parametros2=params2[1]
        datos2 = crear_datos(distr2,parametros2,num_datos - num_drift)
  
        
    elif tipo==7: 
        c = params1[0]
        desv = params1[1]
        s = params1[2]
        phi = params1[3]
        teta = params1[4]
        a = params1[5]
        datos1 = creacion_modelos_ARMA(c,num_drift,desv,s,phi,teta,a)

        tipo2 = params2[0]
        distr,param,p = params2[1],params2[2],params2[3]
        if tipo2==1:
            datos2=datos_periodicos_amplitud(distr,param,num_datos-num_drift,p)
        elif tipo2==2:
            datos2=datos_periodicos_cantidad(distr,param,num_datos-num_drift,p)
            
    elif tipo==8:
        
        c = params1[0]
        desv = params1[1]
        s = params1[2]
        phi = params1[3]
        teta = params1[4]
        a = params1[5]
        datos1 = creacion_modelos_ARMA(c,num_drift,desv,s,phi,teta,a)
    
        param,tipo2,coef_error = params2[0],params2[1],params2[2]
        datos2 = tendencia_det(param,tipo2,num_datos-num_drift,coef_error)  
         
    elif tipo==9: 
        tipo1 = params1[0]
        distr,params,p = params1[1],params1[2],params1[3]
        if tipo1==1:
            datos1=datos_periodicos_amplitud(distr,params,num_drift,p)
        elif tipo1==2:
            datos1=datos_periodicos_cantidad(distr,params,num_drift,p)
            
        tipo2 = params2[0]
        distr,param,p = params2[1],params2[2],params2[3]
        if tipo2==1:
            datos2=datos_periodicos_amplitud(distr,param,num_datos-num_drift,p)
        elif tipo2==2:
            datos2=datos_periodicos_cantidad(distr,param,num_datos-num_drift,p)
    
    elif tipo==10:
        tipo1 = params1[0]
        distr,params,p = params1[1],params1[2],params1[3]
        if tipo1==1:
            datos1=datos_periodicos_amplitud(distr,params,num_drift,p)
        elif tipo1==2:
            datos1=datos_periodicos_cantidad(distr,params,num_drift,p)
            
        distr2=params2[0]
        parametros2=params2[1]
        datos2 = crear_datos(distr2,parametros2,num_datos - num_drift)   
        
    elif tipo == 11:
        tipo1 = params1[0]
        distr,params,p = params1[1],params1[2],params1[3]
        if tipo1==1:
            datos1=datos_periodicos_amplitud(distr,params,num_drift,p)
        elif tipo1==2:
            datos1=datos_periodicos_cantidad(distr,params,num_drift,p)
            
        c2 = params2[0]
        desv2 = params2[1]
        s2 = params2[2]
        phi2 = params2[3]
        teta2 = params2[4]
        a2 = params2[5]
        datos2 = creacion_modelos_ARMA(c2,num_datos-num_drift,desv2,s2,phi2,teta2,a2)  
          
    elif tipo==12:
        tipo1 = params1[0]
        distr,params,p = params1[1],params1[2],params1[3]
        if tipo1==1:
            datos1=datos_periodicos_amplitud(distr,params,num_drift,p)
        elif tipo1==2:
            datos1=datos_periodicos_cantidad(distr,params,num_drift,p)
            
        param,tipo2,coef_error = params2[0],params2[1],params2[2]
        datos2=tendencia_det(param,tipo2,num_datos-num_drift,coef_error)  
           
    elif tipo==13:
        params,tipo1,coef_error = params1[0],params1[1],params1[2]
        datos1=tendencia_det(params,tipo1,num_drift,coef_error) 
        
        param,tipo2,coef_error2 = params2[0],params2[1],params2[2]
        datos2=tendencia_det(param,tipo2,num_datos-num_drift,coef_error2) 
         
    elif tipo==14:
        params,tipo1,coef_error = params1[0],params1[1],params1[2]
        datos1=tendencia_det(params,tipo1,num_drift,coef_error) 
        
        distr2=params2[0]
        parametros2=params2[1]
        datos2 = crear_datos(distr2,parametros2,num_datos - num_drift)
        
    elif tipo==15:
        params,tipo1,coef_error = params1[0],params1[1],params1[2]
        datos1=tendencia_det(params,tipo1,num_drift,coef_error) 
        
        c2 = params2[0]
        desv2 = params2[1]
        s2 = params2[2]
        phi2 = params2[3]
        teta2 = params2[4]
        a2 = params2[5]
        datos2 = creacion_modelos_ARMA(c2,num_datos-num_drift,desv2,s2,phi2,teta2,a2)
        
    elif tipo==16: 
        params,tipo1,coef_error = params1[0],params1[1],params1[2]
        datos1=tendencia_det(params,tipo1,num_drift,coef_error) 
        
        tipo2 = params2[0]
        distr,param,p = params2[1],params2[2],params2[3]
        if tipo2==1:
            datos2=datos_periodicos_amplitud(distr,param,num_datos-num_drift,p)
        elif tipo2==2:
            datos2=datos_periodicos_cantidad(distr,param,num_datos-num_drift,p)
             
    datos = np.concatenate((datos1,datos2))
    return datos 

CREACIÓN DE DATAFRAMES

SEGÚN UN MODELO DE TENDENCIA DETERMINISTA

Parámetros: 
    - params: indica los parámetros de la tendencia.
    - tipo: Valores posibles:
        1. Tendencia determinista lineal
        2. Tendencia determinista polinómica
        3. Tendencia determinista exponencial
        4. Tendencia determinista logaritmica
    - inicio, fin: fecha inicio y fin de la serie temporal.
    - columna: nombre de la columna
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds
    - coef_error: indica el coeficiente de error (a mayor coeficiente, menor estacionariedad). Si el coef_error=0, los datos son estacionarios. 

In [24]:
def crear_df_fin_tend_det(inicio,fin,freq,columna,params,tipo,coef_error=0):
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    datos = tendencia_det(params,tipo,num_datos,coef_error)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df

Parámetros: 
    - params: indica los parámetros de la tendencia.
    - tipo: Valores posibles:
        1. Tendencia determinista lineal
        2. Tendencia determinista cuadrática
        3. Tendencia determinista exponencial
    - inicio, periodos: fecha inicio y nº de periodos.
    - columna: nombre de la columna 
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds
    - coef_error: indica el coeficiente de error (a mayor coeficiente, menor estacionariedad). Si el coef_error=0, los datos son estacionarios. 

In [25]:
def crear_df_periodos_tend_det(inicio,periodos,freq,columna,params,tipo,coef_error=0):
    indice = series_periodos(inicio,periodos,freq)
    num_datos = indice.size
    datos = tendencia_det(params,tipo,num_datos,coef_error)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df

SEGÚN EL MODELO DE DISTRIBUCIONES DE CREAR_DATOS:

Parámetros:
- inicio, fin: fecha inicio y fin de la serie temporal.
- columna: nombre de la columna
- freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds
- Distr, params : Indica la distribución a seguir y los parámetros de esta distribución. Casos:
    1. Los parámetros son la media y la desv_típica de los datos. Sigue una distribución normal.
    2. Los parámetros son n (nº pruebas) y p (probabilidad éxito) de una distribución binomial. Además, podemos añadir otro parámetro que indica la localización de los datos.
    3. El parámetro es mu de una distribución poisson. Además, podemos añadir otro parámetro que indica la localización de los datos.
    4. El parámetro es p (probabilidad éxito) de una distribución geométrica. Además, podemos añadir otro parámetro que indica la localización de los datos.
    5. Los parámetros son M (tamaño población), n (nº aciertos) y N (tamaño muestra) de una distribución hipergeométrica. Además, podemos añadir otro parámetro que indica la localización de los datos.
    6. El parámetro es el valor constante que toma los datos.
    7. Sigue una distribución uniforme. Puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    8. El parámetro es s de una distribución lognormal. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    9. Sigue una distribución exponencial. Puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    10. El parámetro es a de una distribución gamma. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    11. Los parámetros son a y b de una distribución beta. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    12. El parámetro es df de una distribución chi cuadrado. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    13. El parámetro es t de una distribución t-student. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    14. El parámetro es b de una distribución pareto. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    15. Sigue una distribución lineal descendente cuyos parámetros son el dato inicial y la pendiente (en valor absoluto). El mínimo valor posible es cero, si se alcanza los datos se mantienen constante con ese valor. 
    16. Sigue una distribución lineal ascendente cuyos parámetros son el dato inicial y la pendiente. 
    17. Obtienes números aleatorios cuyos valores oscilan entre el primer parámetro y el segundo.

In [26]:
def crear_df_fin_datos(inicio,fin,freq,columna,distr,params):
    
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    datos = crear_datos(distr,params,num_datos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df 

Parámetros:
- inicio, periodos: fecha inicio y nº periodos de la serie temporal. 
- columna: nombre de la columna
- freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds
- Distr, params : Indica la distribución a seguir y los parámetros de esta distribución. Casos:
    1. Los parámetros son la media y la desv_típica de los datos. Sigue una distribución normal.
    2. Los parámetros son n (nº pruebas) y p (probabilidad éxito) de una distribución binomial. Además, podemos añadir otro parámetro que indica la localización de los datos.
    3. El parámetro es mu de una distribución poisson. Además, podemos añadir otro parámetro que indica la localización de los datos.
    4. El parámetro es p (probabilidad éxito) de una distribución geométrica. Además, podemos añadir otro parámetro que indica la localización de los datos.
    5. Los parámetros son M (tamaño población), n (nº aciertos) y N (tamaño muestra) de una distribución hipergeométrica. Además, podemos añadir otro parámetro que indica la localización de los datos.
    6. El parámetro es el valor constante que toma los datos.
    7. Sigue una distribución uniforme. Puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    8. El parámetro es s de una distribución lognormal. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    9. Sigue una distribución exponencial. Puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    10. El parámetro es a de una distribución gamma. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    11. Los parámetros son a y b de una distribución beta. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    12. El parámetro es df de una distribución chi cuadrado. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    13. El parámetro es t de una distribución t-student. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    14. El parámetro es b de una distribución pareto. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    15. Sigue una distribución lineal descendente cuyos parámetros son el dato inicial y la pendiente (en valor absoluto). El mínimo valor posible es cero, si se alcanza los datos se mantienen constante con ese valor. 
    16. Sigue una distribución lineal ascendente cuyos parámetros son el dato inicial y la pendiente. 
    17. Obtienes números aleatorios cuyos valores oscilan entre el primer parámetro y el segundo.

In [27]:
def crear_df_periodos_datos(inicio,periodos,freq,columna,distr,params):
    
    indice = series_periodos(inicio,periodos,freq)
    datos = crear_datos(distr,params,periodos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df 

SEGÚN MODELOS PERIÓDICOS:

Parámetros: 
    - inicio, fin: fecha de inicio y fin de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna 
    - distr, params: indica la distribución y los parámetros que siguen los datos
    - p: indica la cantidad de periodos/ la amplitud de los periodos
    - tipo: valores posibles
        1. p = amplitud de periodo
        2. p = número de periodos

In [28]:
def crear_df_fin_periodicos(inicio,fin,freq,columna,distr,params,p,tipo):
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    if tipo==1:
        datos = datos_periodicos_amplitud(distr,params,num_datos,p)
    elif tipo ==2:
        datos=datos_periodicos_cantidad(distr,params,num_datos,p)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df 

Parámetros: 
    - inicio, periodos: fecha de inicio y nº de periodos de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna 
    - distr, params: indica la distribución y los parámetros que siguen los datos
    - p: indica la cantidad de periodos/ la amplitud de los periodos
    - tipo: valores posibles
        1. p = amplitud de periodo
        2. p = número de periodos

In [39]:
def crear_df_periodos_periodicos(inicio,periodos,freq,columna,distr,params,p,tipo):
    indice = series_periodos(inicio,periodos,freq)
    num_datos = indice.size
    if tipo==1:
        datos = datos_periodicos_amplitud(distr,params,num_datos,p)
    elif tipo==2:
        datos=datos_periodicos_cantidad(distr,params,num_datos,p)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df 

SEGÚN MODELOS AUTORREGRESIVOS Y DE MEDIAS MÓVILES: 

Parámetros: 
    - c, phi, teta, a : parámetros del modelo de autorregresivo y de medias móviles.
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - desv: necesario para la creación de a en el caso de que pasemos como parámetro un array vacío
    - inicio, fin: indica la fecha de inicio y fin de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna 

In [30]:
def crear_df_fin_ARMA(inicio,fin,freq,columna,c,desv,s=0,phi=[],teta=[],a=[]):
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    datos=creacion_modelos_ARMA(c,num_datos,desv,s,phi,teta,a)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df 

Parámetros:
    - c, phi, teta, a : parámetros del modelo de autorregresivo y de medias móviles. 
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - desv: necesarios para la creación de a en el caso de que pasemos como parámetro un array vacío
    - inicio, periodos: indica la fecha de inicio y nº de periodos de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna

In [31]:
def crear_df_periodos_ARMA(inicio,periodos,freq,columna,c,desv,s=0,phi=[],teta=[],a=[]):
    indice = series_periodos(inicio,periodos,freq)
    datos=creacion_modelos_ARMA(c,periodos,desv,s,phi,teta,a)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df 

MODELOS CON DRIFT 

Parámetros:
    - params1,params2 : parámetros necesarios para crear los datos pervios y posteriores al drift. Casos:
        1. DISTRIBUCIONES CREADAS CON CREAR DATOS: los parámetros son distr y parametros que indica la distribución y los parámetros de la distribución.
        2. MODELOS ARMA: los parámetros son los del modelo arma,es decir, c, phi, teta, a, s y desv.
        3. DATOS PERIODICOS: los parametros son el tipo (1 indica que se ha pasado la amplitud del periodo y 2 la cantidad de periodos) y distr, params y p (indica el número de periodos o su amplitud) necesarios para el crear los datos periódicos.
        4. TENDENCIA DETERMINISTA: los parámetros son el tipo de tendencia, los parámetros del modelo y el coeficiente de error.  
    - tipo: tipo de drift.
    - num_drift: número de datos previos al drift.
    - inicio,fin: indica la fecha de inicio y fin de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna

In [32]:
def crear_df_fin_DRIFT(inicio,fin,freq,columna,params1,params2,tipo,num_drift):
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    datos = crear_drift(params1,params2,tipo,num_drift,num_datos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df 

Parámetros:
    - params1,params2 : parámetros necesarios para crear los datos pervios y posteriores al drift. Casos:
        1. DISTRIBUCIONES CREADAS CON CREAR DATOS: los parámetros son distr y parametros que indica la distribución y los parámetros de la distribución.
        2. MODELOS ARMA: los parámetros son los del modelo arma,es decir, c, desv, s, phi, teta, a.
        3. DATOS PERIODICOS: los parametros son el tipo (1 indica que se ha pasado la amplitud del periodo y 2 la cantidad de periodos) y distr, params y p (indica el número de periodos o su amplitud) necesarios para el crear los datos periódicos.
        4. TENDENCIA DETERMINISTA: los parámetros son el tipo de tendencia, los parámetros del modelo y el coeficiente de error.  
    - tipo: tipo de drift.
    - num_drift: número de datos previos al drift.
    - inicio, periodos: indica la fecha de inicio y nº de periodos de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna

In [33]:
def crear_df_periodos_DRIFT(inicio,periodos,freq,columna,params1,params2,tipo,num_drift):
    indice = series_periodos(inicio,periodos,freq)
    datos = crear_drift(params1,params2,tipo,num_drift,periodos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df 

GENERALIZACIÓN: 

Parámetros:
    - inicio, fin: indica la fecha de inicio y fin de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna
    - tipo, parametros: 
        1. Distribucion: los parámetros son distr, params que indica el tipo de distribución y los parámetros de esta distribución.
        2. ARMA: los parámetros son c (constante), desv (desviación del ruido blanco), s (estacionalidad), phi, teta(parámetros que participan en el modelo), a(ruido blanco)
        3. Periódicos: los parámetros son el tipo de modelo periódico, distr, params (distribución que seguirán los datos y sus parámetros), p (indica la cantidad de periodos o la amplitud de periodo)
        4. Tendencia determinista: los parámetros son params (parámetros del modelo de tendencia determinista), tipo (tipo de tendencia: lineal, polinómica, exponencial o logarítmica), coef_error 
        5. Drift: los parámetros son params1,params2 (parámetros necesarios para crear los datos pervios y posteriores al drift, es decir, los indicados en los otros tipos), tipo_drift (tipo de drift) y num_drift(número de datos previos al drift).

In [32]:
def crear_df_fin(inicio,fin,freq,columna,parametros,tipo):
    
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    
    if tipo==1: # DISTRIBUCIÓN
        distr = parametros[0]
        params = parametros[1]
        datos = crear_datos(distr,params,num_datos)
        
    elif tipo ==2: # ARMA
        c = parametros[0]
        desv = parametros[1]
        s = parametros[2]
        phi = parametros[3]
        teta = parametros[4]
        a = parametros[5]
        datos = creacion_modelos_ARMA(c,num_datos,desv,s,phi,teta,a)
        
    elif tipo == 3: # MODELOS PERIÓDICOS
        tipo_per=parametros[0]
        distr = parametros[1]
        params = parametros[2]
        p = parametros[3]
        if tipo_per==1:
            datos = datos_periodicos_amplitud(distr,params,num_datos,p)
        elif tipo_per==2:
            datos = datos_periodicos_cantidad(distr,params,num_datos,p)
            
    elif tipo ==4: # TENDENCIA DETERMINISTA     
        params = parametros[0]
        tipo_tend = parametros[1]
        coef_error = parametros[2]
        datos = tendencia_det(params,tipo_tend,num_datos,coef_error)
    
    elif tipo == 5 : #DRIFT 
        params1 = parametros[0] 
        params2 = parametros[1]
        tipo_drift = parametros[2]
        num_drift = parametros[3]
        datos = crear_drift(params1,params2,tipo_drift,num_drift,num_datos)
        
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df

Parámetros:
    - inicio, periodos: indica la fecha de inicio y número de periodos de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna
    - tipo, parametros: 
        1. Distribucion: los parámetros son distr, params que indica el tipo de distribución y los parámetros de esta distribución.
        2. ARMA: los parámetros son c (constante), desv (desviación del ruido blanco), s (estacionalidad), phi, teta(parámetros que participan en el modelo), a(ruido blanco)
        3. Periódicos: los parámetros son el tipo de modelo periódico, distr, params (distribución que seguirán los datos y sus parámetros), p (indica la cantidad de periodos o la amplitud de periodo)
        4. Tendencia determinista: los parámetros son params (parámetros del modelo de tendencia determinista), tipo (tipo de tendencia: lineal, polinómica, exponencial o logarítmica), coef_error 
        5. Drift: los parámetros son params1,params2 (parámetros necesarios para crear los datos pervios y posteriores al drift, es decir, los indicados en los otros tipos), tipo_drift (tipo de drift) y num_drift(número de datos previos al drift).

In [33]:
def crear_df_periodos(inicio,periodos,freq,columna,parametros,tipo):
    
    indice = series_periodos(inicio,periodos,freq)
    num_datos = indice.size
    
    if tipo==1: # DISTRIBUCIÓN
        distr = parametros[0]
        params = parametros[1]
        datos = crear_datos(distr,params,num_datos)
        
    elif tipo ==2: # ARMA
        c = parametros[0]
        desv = parametros[1]
        s = parametros[2]
        phi = parametros[3]
        teta = parametros[4]
        a = parametros[5]
        datos = creacion_modelos_ARMA(c,num_datos,desv,s,phi,teta,a)
        
    elif tipo == 3: # MODELOS PERIÓDICOS
        tipo_per=parametros[0]
        distr = parametros[1]
        params = parametros[2]
        p = parametros[3]
        if tipo_per==1:
            datos = datos_periodicos_amplitud(distr,params,num_datos,p)
        elif tipo_per==2:
            datos = datos_periodicos_cantidad(distr,params,num_datos,p)
            
    elif tipo ==4: # TENDENCIA DETERMINISTA     
        params = parametros[0]
        tipo_tend = parametros[1]
        coef_error = parametros[2]
        datos = tendencia_det(params,tipo_tend,num_datos,coef_error)
    
    elif tipo == 5 : #DRIFT 
        params1 = parametros[0] 
        params2 = parametros[1]
        tipo_drift = parametros[2]
        num_drift = parametros[3]
        datos = crear_drift(params1,params2,tipo_drift,num_drift,num_datos)
        
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df

EXPORTACIÓN DE DATOS DE UN DATAFRAME A UN CSV Y VICEVERSA

Parámetros:
    - df: dataframe 
    - folder: nombre de la carpeta donde se guardarán los datos 
    - file: nombre del archivo donde se guardarán los datos

In [34]:
def csv_df(df,folder,file):
    filepath = Path(folder + '/'+ file +'.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df.to_csv(filepath,index_label='indice')

Parámetros:
    - folder: nombre de la carpeta donde se guardan los datos
    - file: nombre del archivo donde se gardan los datos
    - indice: nombre de la columna que se usa como indice

In [35]:
def df_csv(folder,file,indice):
    return pd.read_csv(folder+'/'+file+'.csv',index_col=indice)

DATAFRAME CON VARIAS COLUMNAS: 

Parámetros: 
    - inicio, fin: indica la fecha de inicio y final de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columnas: array con el nombres de la columnas 
    - parameters: array donde en cada posición guarda un array con los parametros necesario para cada columna según el tipo.
    - tipos: array donde en cada posición guarda el tipo de modelo que siguen los datos de la columna. Casos:  
        1. Distribución: los parámetros son distr, params que indica el tipo de distribución y los parámetros de esta distribución.
        2. ARMA: los parámetros son c (constante), desv (desviación del ruido blanco), s (estacionalidad), phi, teta(parámetros que participan en el modelo), a(ruido blanco)
        3. Periódicos: los parámetros son el tipo de modelo periódico, distr, params (distribución que seguirán los datos y sus parámetros), p (indica la cantidad de periodos o la amplitud de periodo)
        4. Tendencia determinista: los parámetros son params (parámetros del modelo de tendencia determinista), tipo (tipo de tendencia: lineal, polinómica, exponencial o logarítmica), coef_error 
        5. Drift: los parámetros son params1,params2 (parámetros necesarios para crear los datos pervios y posteriores al drift, es decir, los indicados en los otros tipos), tipo_drift (tipo de drift) y num_drift(número de datos previos al drift).


In [4]:
def crear_df_multi_fin(inicio,fin,freq,columnas,parameters,tipos):
    
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    
    for k in range(0, len(tipos)): 
        
        tipo = tipos[k]  
        parametros = parameters[k]
        columna = columnas[k]
        
        if tipo==1: # DISTRIBUCIÓN
            distr = parametros[0]
            params = parametros[1]
            datos_n = crear_datos(distr,params,num_datos)
            
        elif tipo ==2: # ARMA
            c = parametros[0]
            desv = parametros[1]
            s = parametros[2]
            phi = parametros[3]
            teta = parametros[4]
            a = parametros[5]
            datos_n = creacion_modelos_ARMA(c,num_datos,desv,s,phi,teta,a)
            
        elif tipo == 3: # MODELOS PERIÓDICOS
            tipo_per=parametros[0]
            distr = parametros[1]
            params = parametros[2]
            p = parametros[3]
            if tipo_per==1:
                datos_n = datos_periodicos_amplitud(distr,params,num_datos,p)
            elif tipo_per==2:
                datos_n = datos_periodicos_cantidad(distr,params,num_datos,p)
                
        elif tipo ==4: # TENDENCIA DETERMINISTA     
            params = parametros[0]
            tipo_tend = parametros[1]
            coef_error = parametros[2]
            datos_n = tendencia_det(params,tipo_tend,num_datos,coef_error)
        
        elif tipo == 5 : # DRIFT 
            params1 = parametros[0] 
            params2 = parametros[1]
            tipo_drift = parametros[2]
            num_drift = parametros[3]
            datos_n = crear_drift(params1,params2,tipo_drift,num_drift,num_datos)
            
        if k==0:
            df = pd.DataFrame(data=datos_n,index=indice,columns=columna)
        else :     
            df_n = pd.DataFrame(data=datos_n,index=indice,columns=columna)
            df= df.join(df_n, how="outer")
        
    
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df

Parámetros: 
    - inicio, periodos: indica la fecha de inicio y nº de periodos de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columnas: array con el nombres de la columnas 
    - parameters: array donde en cada posición guarda un array con los parametros necesario para cada columna según el tipo.
    - tipos: array donde en cada posición guarda el tipo de modelo que siguen los datos de la columna. Casos:  
        1. Distribución: los parámetros son distr, params que indica el tipo de distribución y los parámetros de esta distribución.
        2. ARMA: los parámetros son c (constante), desv (desviación del ruido blanco), s (estacionalidad), phi, teta(parámetros que participan en el modelo), a(ruido blanco)
        3. Periódicos: los parámetros son el tipo de modelo periódico, distr, params (distribución que seguirán los datos y sus parámetros), p (indica la cantidad de periodos o la amplitud de periodo)
        4. Tendencia determinista: los parámetros son params (parámetros del modelo de tendencia determinista), tipo (tipo de tendencia: lineal, polinómica, exponencial o logarítmica), coef_error 
        5. Drift: los parámetros son params1,params2 (parámetros necesarios para crear los datos pervios y posteriores al drift, es decir, los indicados en los otros tipos), tipo_drift (tipo de drift) y num_drift (número de datos previos al drift).


In [1]:
def crear_df_multi_periodos(inicio,periodos,freq,columnas,parameters,tipos):
    
    indice = series_periodos(inicio,periodos,freq)
    num_datos = indice.size
    
    for k in range(0, len(tipos)): 
        
        tipo = tipos[k]  
        parametros = parameters[k]
        
        if tipo==1: # DISTRIBUCIÓN
            distr = parametros[0]
            params = parametros[1]
            datos_n = crear_datos(distr,params,num_datos)
            
        elif tipo ==2: # ARMA
            c = parametros[0]
            desv = parametros[1]
            s = parametros[2]
            phi = parametros[3]
            teta = parametros[4]
            a = parametros[5]
            datos_n = creacion_modelos_ARMA(c,num_datos,desv,s,phi,teta,a)
            
        elif tipo == 3: # MODELOS PERIÓDICOS
            tipo_per=parametros[0]
            distr = parametros[1]
            params = parametros[2]
            p = parametros[3]
            if tipo_per==1:
                datos_n = datos_periodicos_amplitud(distr,params,num_datos,p)
            elif tipo_per==2:
                datos_n = datos_periodicos_cantidad(distr,params,num_datos,p)
                
        elif tipo ==4: # TENDENCIA DETERMINISTA     
            params = parametros[0]
            tipo_tend = parametros[1]
            coef_error = parametros[2]
            datos_n = tendencia_det(params,tipo_tend,num_datos,coef_error)
        
        elif tipo == 5 : # DRIFT 
            params1 = parametros[0] 
            params2 = parametros[1]
            tipo_drift = parametros[2]
            num_drift = parametros[3]
            datos_n = crear_drift(params1,params2,tipo_drift,num_drift,num_datos)
        
        if k==0:
            datos = np.array(datos_n).reshape(1,-1)
        else : 
            datos = np.concatenate(datos,np.array(datos_n).reshape(1,-1))
            
    df = pd.DataFrame(data=datos,index=indice,columns=columnas)
    df.plot(title='Serie Temporal',figsize=(13,5))
    return df